# Getting Started with the AppEEARS API: Submitting and Downloading an Area Request
### This tutorial demonstrates how to use Python to connect to the AppEEARS API 
The Application for Extracting and Exploring Analysis Ready Samples ([AppEEARS](https://lpdaacsvc.cr.usgs.gov/appeears/)) offers a simple and efficient way to access and transform geospatial data from a variety of federal data archives in an easy-to-use web application interface. 

The [AppEEARS API](https://lpdaacsvc.cr.usgs.gov/appeears/api/) offers users **programmatic access** to all features available in AppEEARS, with the exception of visualizations. The API features are demonstrated in this notebook.  
***
### Example: Submit an area request with the continental United States as the region of interest for extracting land surface temperature data.    
AppEEARS area sample requests allow users to subset their desired data by spatial area via vector polygons (shapefiles or GeoJSONs). Users can also reproject and reformat the output data. AppEEARS returns the valid data from the parameters defined within the sample request.    

#### Data Used in the Example:
- Data layers:   
    - ECOSTRESS Land Surface Temperature Daily 70m  
        - [ECO2LSTE.001],   
***  
# Table of Contents:
1. **Getting Started/Pre Reqs**  
    1a. Enable Access to the API  
    1b. Set Up the Working Environment    
    1c. Login  
  
3. **Submit an Area Request [Tasks]**  
    3a. Import a Shapefile  
    3b. Search and Explore Available Projections [Spatial API]   
    3c. Compile a JSON [Task Object]  
    3d. Submit a Task Request [Submit Task]  
    3e. Retrieve Task Status [Retrieve Task]  
    
4. **Download a Request [Bundle API]**    
    4a. Explore Files in Request Output [List Files]    
    4b. Download all Files in a Request [Download File]   
  
***
### Dependencies:
- This tutorial was tested using Python 3.6.1.  
- A [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to complete this tutorial. You can create an account at the link provided.    
- To execute section 6, the [Geospatial Data Abstraction Library](http://www.gdal.org/) (GDAL) is required. 
***
### AppEEARS Information:
To access AppEEARS, visit: https://lpdaacsvc.cr.usgs.gov/appeears/  
> For comprehensive documentation of the full functionality of the AppEEARS API, please see the AppEEARS [API Documentation](https://lpdaacsvc.cr.usgs.gov/appeears/api/) 


***

### Source Code used to Generate this Tutorial: UPDATE LATER
- [Jupyter Notebook](https://git.earthdata.nasa.gov/projects/LPDUR/repos/cole-sandbox/browse/.ipynb_checkpoints/AppEEARS_GeoTIFF_TS-checkpoint.ipynb) 

***

# 1. Getting Started
<div class="alert alert-block alert-warning" >
<b>NOTE:</b> If this is your first time using the AppEEARS API, you must first enable API access by following the instructions provided below.</div> 

***
##  Enable Access to the API
#### AppEEARS API access requires the same [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) as the AρρEEARS user interface. In addition to having a valid NASA Earthdata Login account, the API feature must be enabled for the user within AρρEEARS.
#### To enable access to the AppEEARS API, navigate to the [AppEEARS website](https://lpdaacsvc.cr.usgs.gov/appeears/). Click the *Sign In* button in the top right portion of the AppEEARS landing page screen.  
![AppEEARS Sign In](https://lpdaacsvc.cr.usgs.gov/assets/images/help/image001.73bb446d.png)  
#### Once you are signed in, click the *Manage User* icon in the top right portion of the AppEEARS landing page screen and select *Settings*. 
![Manage User -> Settings](https://lpdaacsvc.cr.usgs.gov/assets/images/help/api/image001.30a681c0.png)      
#### Select the *Enable API* box to gain access to the AppEEARS API.  
![Enable API Access](https://lpdaacsvc.cr.usgs.gov/assets/images/help/api/image002.2262e4d3.png)

***
## Set Up the Working Environment
#### Next, import the required packages, set the input/working directory, and create an output directory for the results.

In [1]:
# Import packages
import requests as r
import getpass, pprint, time, os, cgi

#### If you are missing any of the packages above, download them in order to use the full functionality of this tutorial. 

In [2]:
# Set input directory, change working directory
inDir = '/home/osboxes/inDir'            # IMPORTANT: Update to dir on your OS
os.chdir(inDir)                                                                 # Change to working directory

# below, set the AppEEARS API to a variable (this URL also will take you to the AppEEARS API documentation)
api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'

<div class="alert alert-block alert-warning" >
<b>If you plan to execute this tutorial on your own OS, `inDir` above needs to be changed.</b>
</div> 

***
## Login
#### To submit a request, you must first login to the AppEEARS API. Use the `getpass` package to enter your NASA Earthdata login **Username** and **Password**. When prompted after executing the code block below, enter your username followed by your password.

In [3]:
user = 'skylinesgis'      # Input NASA Earthdata Login Username
password = '12FUCKthisshit!'  # Input NASA Earthdata Login Password

#### Use the `requests` package to post your username and password. A successful login will provide you with a token to be used later in this tutorial to submit a request. For more information or if you are experiencing difficulties, please see the [API Documentation](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#login)

In [4]:
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert base API URL, call the login service, provide credentials & return json
del user, password                                                           # Remove user and password information upon a successful login 
token_response                                                               # Print response

{'token_type': 'Bearer',
 'token': 'r09sgRLjIqyzUB6BScaLyjU7Gy2QWiarNRvC8RmWcbi03ye9pVQvHbTZmtLs0v8v6410CjA7QKCV3dwEKF-yMQ',
 'expiration': '2019-11-24T10:00:18Z'}

***
## Submit an Area Request [[Tasks](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#tasks)]
#### The **Submit task** API call provides a way to submit a new request to be processed. It can accept data via JSON, query string, or a combination of both. In the example below, compile a json and submit a request. Tasks in AppEEARS correspond to each request associated with your user account. Therefore, each of the calls to this service requires an authentication token (see Section 1C.), which is stored in a header below. 

In [5]:
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request below

## Import a Shapefile
#### In this section, begin by importing a shapefile using the `geopandas` package. The shapefile is publically available for download from the [NPS website](https://irma.nps.gov/DataStore/Reference/Profile/2224545?lnv=True).

In [6]:
try:
    import geopandas as gpd
except:
    !python -m pip install geopandas
    import geopandas as gpd

import json

# Read in shapefile
nps = gpd.read_file('{}/cb_2018_us_nation_5m.shp'.format(inDir)) # UPDATE
print(nps.head())                                                              # Print first few lines of df

    AFFGEOID GEOID           NAME  \
0  0100000US    US  United States   

                                            geometry  
0  MULTIPOLYGON (((179.48246 51.98283, 179.48656 ...  


#### Below, query the geopandas dataframe for the national park that you are interested in using for your region of interest, here *United States of America (continental)

In [7]:
nps_gc = nps.to_json() # Extract GCNP and set to variable
nps_gc = json.loads(nps_gc)                                            # Convert to json format

---
##  Search and Explore Available Projections [[Spatial API](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#spatial)]   
#### The spatial API provides some helper services used to support submitting area task requests. The call below will retrieve the list of supported projections in AppEEARS.

In [8]:
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
projections                                               # Print projections and information

[{'Name': 'native',
  'Description': 'Native Projection',
  'Platforms': '',
  'Proj4': '',
  'Datum': '',
  'EPSG': '',
  'Units': '',
  'GridMapping': '',
  'Available': True},
 {'Name': 'geographic',
  'Description': 'Geographic',
  'Platforms': "['SRTM', 'ECOSTRESS', 'SSEBop ET', 'GPW', 'ASTER GDEM']",
  'Proj4': '+proj=longlat +datum=WGS84 +no_defs',
  'Datum': 'wgs84',
  'EPSG': 4326.0,
  'Units': 'degrees',
  'GridMapping': 'latitude_longitude',
  'Available': True},
 {'Name': 'sinu_modis',
  'Description': 'MODIS Sinusoidal',
  'Platforms': "['Combined MODIS', 'Terra MODIS', 'Aqua MODIS', 'S-NPP NASA VIIRS', 'Global WELD']",
  'Proj4': '+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs',
  'Datum': '',
  'EPSG': '',
  'Units': 'meters',
  'GridMapping': 'sinusoidal',
  'Available': True},
 {'Name': 'albers_weld_alaska',
  'Description': 'WELD Albers Equal Area Alaska',
  'Platforms': "['WELD']",
  'Proj4': '+proj=aea +lat_1=55 +lat_2=65 +lat_0=50

#### Create a dictionary of projections with projection `Name` as the keys.

In [9]:
projs = {}                                  # Create an empty dictionary
for p in projections: projs[p['Name']] = p  # Fill dictionary with `Name` as keys
list(projs.keys())                          # Print dictionary keys

['native',
 'geographic',
 'sinu_modis',
 'albers_weld_alaska',
 'albers_weld_conus',
 'albers_ard_alaska',
 'albers_ard_conus',
 'albers_ard_hawaii',
 'easegrid_2_global',
 'easegrid_2_north',
 'laea_sphere_19']

#### Print the information for the projection used in this tutorial, Geographic.

In [10]:
projs['native']

{'Name': 'native',
 'Description': 'Native Projection',
 'Platforms': '',
 'Proj4': '',
 'Datum': '',
 'EPSG': '',
 'Units': '',
 'GridMapping': '',
 'Available': True}

---
## 3c. Compile a JSON [[Task Object](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#task-object)]
#### In this section, begin by setting up the information needed to compile an acceptable json for submitting an AppEEARS point request. For detailed information on required json parameters, see the [API Documentation](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#tasks). 

In [11]:

from datetime import datetime, timedelta, date
today = date.today()
print("Today's date:", today)
start = datetime.strptime(str(today), "%Y-%m-%d") #string to date
end = start - timedelta(days=10) # date - days
print (start.strftime("%m-%d-%Y"),end.strftime("%m-%d-%Y") )




task_type = ['point','area']        # Type of task, area or  point
task_name = 'lst_daily'                 # user-defined name of the task 'NPS Vegetation Area' 
startDate = start.strftime("%m-%d-%Y")            # Start of the date range for  which to extract data: MM-DD-YYYY
endDate = end.strftime("%m-%d-%Y")              # End of the date range for  which to extract data: MM-DD-YYYY
recurring = False                   # Specify True for a recurring date range
# yearRange = [2000, 2016] if recurring = True, set yearRange and change start and end date to MM-DD
proj = projs['geographic']['Name']  # Set output projection 
outFormat = ['geotiff', 'netcdf4']  # Set output file format type



# {"error": null, "params": {"geo": {"type": "FeatureCollection", 
#                                    "features": [{"type": "Feature", 
#                                                  "geometry": 
#                                                  {"type": "Polygon", 
#                                                   "coordinates": [
#                                                       [[-123.9609375, 24.046875],
#                                                        [-123.9609375, 48.65625], 
#                                                        [-69.8203125, 48.65625],
#                                                        [-69.8203125, 24.046875], 
#                                                        [-123.9609375, 24.046875]]]}, 
#                                                  "properties": {}}], 
#                                    "fileName": "User-Drawn-Polygon"}, 
#                            "dates": [{"endDate": "11-19-2019", "startDate": "11-12-2019", 
#                                       "yearRange": [1980, 2050]}], 
#                            "layers": [{"layer": "SDS_LST", "product": "ECO2LSTE.001"}],
#                            "output": {"format": {"type": "geotiff"}, "projection": "native"}},
#  "status": "processing", "created": "2019-11-20T05:35:28.634090", 
#  "task_id": "fd7ce40b-9148-4414-bbb5-374641f03261", "updated": "2019-11-20T18:38:08.124372", "user_id": "aslan@denverco.us", "retry_at": null, "task_name": "11-19-2019", "task_type": "area", "api_version": null, "svc_version": "2.31", "web_version": "2.31", "expires_on": "2019-12-20T18:38:08.124372", "attempts": 1}

Today's date: 2019-11-22
11-22-2019 11-12-2019


#### Below, compile the JSON to be submitted as a point request. Notice that `nps_gc` is inserted from the shapefile transformed to a json via the `geopandas` and `json` packages above in section 3a. 

In [12]:
task = {
   "task_type":"area",
   "task_name":"kkk",
   "params":{
      "geo":{
         "type":"FeatureCollection",
         "features":[
            {
               "type":"Feature",
               "geometry":{
                  "type":"Polygon",
                  "coordinates":[
                     [
                        [
                           -123.9609375,
                           24.046875
                        ],
                        [
                           -123.9609375,
                           48.65625
                        ],
                        [
                           -69.8203125,
                           48.65625
                        ],
                        [
                           -69.8203125,
                           24.046875
                        ],
                        [
                           -123.9609375,
                           24.046875
                        ]
                     ]
                  ]
               },
               "properties":{

               }
            }
         ],
         "fileName":"User-Drawn-Polygon"
      },
      "dates":[
         {
            "endDate":startDate,
            "startDate":endDate,
            "yearRange":[
               1980,
               2050
            ]
         }
      ],
      "layers":[
         {
            "layer":"SDS_LST",
            "product":"ECO2LSTE.001"
         }
      ],
      "output":{
         "format":{
            "type":outFormat[0]},
         
         "projection":"native"
      }
   }
}


***
## Submit a Task Request [[Submit Task](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#submit-task)]
#### Below, post a call to the API task service, using the `task` json created above.

In [15]:
task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post the json to the API task service, and return the response as a json
task_response                                                                 # Print task response

{'task_id': '3b00de7c-ca7d-4237-aa7f-3a0dd0d029d3', 'status': 'pending'}

***
## 3c. Retrieve Task Status [[Retrieve Task](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#retrieve-task)]
#### This API call will list all of the requests associated with your user account, automatically sorted by date descending with the most recent requests listed first. 

#### The AppEEARS API contains some helpful formatting resources. Below, limit the API response to 4 and set `pretty` to True to format the response as an organized json, making it easier to read. Additional information on AppEEARS API [Pagination](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#pagination) and [formatting](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#formatting) can be found in the API documentation. 

In [16]:
params = {'limit': 4, 'pretty': True} # Limit API response to 4 most recent entries, and return as pretty json

In [17]:
tasks_response = r.get('{}task'.format(api),params = params, headers=head).json() # Query the task service setting the params and header containing user token
tasks_response                                                                    # Print tasks response

[{'params': {'dates': [{'endDate': '11-22-2019',
     'startDate': '11-12-2019',
     'yearRange': [1980, 2050]}],
   'layers': [{'layer': 'SDS_LST', 'product': 'ECO2LSTE.001'}],
   'output': {'format': {'type': 'geotiff'}, 'projection': 'native'}},
  'status': 'pending',
  'created': '2019-11-22T20:22:52.046240',
  'task_id': '3b00de7c-ca7d-4237-aa7f-3a0dd0d029d3',
  'updated': '2019-11-22T20:22:52.061968',
  'user_id': 'aslan@denverco.us',
  'task_name': 'kkk',
  'task_type': 'area',
  'api_version': 'v1',
  'svc_version': '2.31.2',
  'web_version': None,
  'expires_on': '2019-12-22T20:22:52.061968'},
 {'params': {'dates': [{'endDate': '11-22-2019',
     'startDate': '11-12-2019',
     'yearRange': [1980, 2050]}],
   'layers': [{'layer': 'SDS_LST', 'product': 'ECO2LSTE.001'}],
   'output': {'format': {'type': 'geotiff'}, 'projection': 'native'}},
  'status': 'pending',
  'created': '2019-11-22T20:22:24.940896',
  'task_id': 'cf8d0c5d-767b-47a6-996f-57a1ebb9070e',
  'updated': '2019-1

#### Next, take the task id returned from the `task_response` that was generated when submitting your request, and use the AppEEARS API status service to check the status of your request.

In [18]:
task_id = task_response['task_id']                                               # Set task id from request submission
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID and user credentials, return json
status_response                                                                  # Print response

{'task_id': '3b00de7c-ca7d-4237-aa7f-3a0dd0d029d3',
 'status': 'pending',
 'user_id': 'aslan@denverco.us',
 'updated': '2019-11-22T20:22:52.061968',
 'status_type': 'task'}

#### Above, if your request is still processing, you can find information on the status of how close it is to completing. 
#### Below, call the task service for your request every 20 seconds to check the status of your request.

In [ ]:
# Ping API until request is complete, then continue to Section 4
starttime = time.time()
while r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'] != 'done':
    print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])
    time.sleep(20.0 - ((time.time() - starttime) % 20.0))
print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])

pending
pending


***
# 4. Download a Request [[Bundle API](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#bundle)]
#### Before downloading the request output, set up an output directory to store the output files, and examine the files contained in the request output. 

In [ ]:
destDir = os.path.join(inDir, task_name)                # Set up output directory using input directory and task name
if not os.path.exists(destDir):os.makedirs(destDir)     # Create the output directory

---
## 4a. Explore Files in Request Output [[List Files](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#list-files)]
#### The bundle API provides information about completed tasks. For any completed task, a bundle can be queried to return the files contained as a part of the task request. Below, call the bundle API and return all of the output files.

In [ ]:
bundle = r.get('{}bundle/{}'.format(api,task_id)).json()  # Call API and return bundle contents for the task_id as json
bundle                                                    # Print bundle contents

***
## 4b. Download Files in a Request (Automation) [[Download File](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#download-file)]
#### Now, use the contents of the bundle to select the file name and id and store as a dictionary. 

In [ ]:
files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values
files    

#### Use the `files` dictionary and a for loop to automate downloading all of the output files into the output directory. 

In [ ]:
for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f),stream=True)                                 # Get a stream to the bundle file
    filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])  # Parse the filename from the Content-Disposition header 
    filepath = os.path.join(destDir, filename)                                                       # Create output file path
    with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir))